<a href="https://colab.research.google.com/github/p2o/chatbot-unicamp-training/blob/master/bert_base_squad_tpu_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 49.6 MB/s 
     |████████████████████████████████| 6.6 MB 37.9 MB/s 
     |████████████████████████████████| 101 kB 8.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.4 MB/s 
     |████████████████████████████████| 212 kB 49.5 MB/s 
     |████████████████████████████████| 140 kB 45.3 MB/s 
     |████████████████████████████████| 1.1 MB 50.0 MB/s 
     |████████████████████████████████| 127 kB 53.5 MB/s 
     |████████████████████████████████| 271 kB 54.0 MB/s 
     |████████████████████████████████| 94 kB 847 kB/s 
     |██████████████

In [ ]:
VERSION = "1.8.1"
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6034  100  6034    0     0  29009      0 --:--:-- --:--:-- --:--:-- 29009
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-1.8.1 ...
Found existing installation: torch 1.12.0+cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 2.6 MB/s 
Uninstalling torch-1.12.0+cu113:
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.11
    Uninstalling google-api-python-client-1.12.11:
      Successfully uninstalled google-api-python-client-1.12.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.316 requires goo

In [ ]:
squad_v2 = False
model_checkpoint = "neuralmind/bert-base-portuguese-cased"
batch_size = 4

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
# Get dataset SQUAD in Portuguese
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Q0IaIlv2h2BC468MwUFmUST0EyN7gNkn' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Q0IaIlv2h2BC468MwUFmUST0EyN7gNkn" -O squad-pt.tar.gz && rm -rf /tmp/cookies.txt


--2022-07-17 23:35:51--  https://docs.google.com/uc?export=download&confirm=t&id=1Q0IaIlv2h2BC468MwUFmUST0EyN7gNkn
Resolving docs.google.com (docs.google.com)... 173.194.192.113, 173.194.192.139, 173.194.192.101, ...
Connecting to docs.google.com (docs.google.com)|173.194.192.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-20-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i181319k0c3mt89ajer6rpiscn8uobkp/1658100900000/03445611175480770093/*/1Q0IaIlv2h2BC468MwUFmUST0EyN7gNkn?e=download&uuid=25fed1d1-c46e-4fd2-8748-48c34938fd83 [following]
--2022-07-17 23:35:51--  https://doc-0s-20-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i181319k0c3mt89ajer6rpiscn8uobkp/1658100900000/03445611175480770093/*/1Q0IaIlv2h2BC468MwUFmUST0EyN7gNkn?e=download&uuid=25fed1d1-c46e-4fd2-8748-48c34938fd83
Resolving doc-0s-20-docs.googleusercontent.com (doc-0s-20-docs.googleusercontent.com)... 108.177.121.

In [ ]:
!tar -xvf squad-pt.tar.gz

squad-train-v1.1.json
squad-dev-v1.1.json


In [ ]:
%%time
# new

# Get the train and validation json file in the HF script format
# inspiration: file squad.py at https://github.com/huggingface/datasets/tree/master/datasets/squad

import json

files = ['squad-train-v1.1.json','squad-dev-v1.1.json']

for file in files:

    # Opening JSON file & returns JSON object as a dictionary
    f = open(file, encoding="utf-8")
    data = json.load(f)

    # Iterating through the json list
    entry_list = list()
    id_list = list()

    for row in data['data']:
        title = row['title']

        for paragraph in row['paragraphs']:
            context = paragraph['context']

            for qa in paragraph['qas']:
                entry = {}

                qa_id = qa['id']
                question = qa['question']
                answers = qa['answers']

                entry['id'] = qa_id
                entry['title'] = title.strip()
                entry['context'] = context.strip()
                entry['question'] = question.strip()

                answer_starts = [answer["answer_start"] for answer in answers]
                answer_texts = [answer["text"].strip() for answer in answers]
                entry['answers'] = {}
                entry['answers']['answer_start'] = answer_starts
                entry['answers']['text'] = answer_texts

                entry_list.append(entry)

    reverse_entry_list = entry_list[::-1]

    # for entries with same id, keep only last one (corrected texts by he group Deep Learning Brasil)
    unique_ids_list = list()
    unique_entry_list = list()
    for entry in reverse_entry_list:
        qa_id = entry['id']
        if qa_id not in unique_ids_list:
            unique_ids_list.append(qa_id)
            unique_entry_list.append(entry)

    # Closing file
    f.close()

    new_dict = {}
    new_dict['data'] = unique_entry_list

    file_name = 'pt_' + str(file)
    with open(file_name, 'w') as json_file:
        json.dump(new_dict, json_file)

CPU times: user 4min 58s, sys: 1.8 s, total: 5min
Wall time: 5min 7s


In [ ]:
from datasets import load_dataset, load_metric
datasets = load_dataset('json',
                        data_files={'train': 'pt_squad-train-v1.1.json', 'validation': 'pt_squad-dev-v1.1.json'},
                        field='data')

Using custom data configuration default-6ff636cc5140e674


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-6ff636cc5140e674/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,5727fec0ff5b5019007d9a72,Culture,"Outros críticos do século XIX, seguindo Rousseau, aceitaram essa diferenciação entre cultura superior e inferior, mas viram o refinamento e a sofisticação da alta cultura como desenvolvimentos corruptos e não naturais que obscurecem e distorcem a natureza essencial das pessoas. Esses críticos consideravam a música folclórica (como produzida pelo ""povo"", isto é, rural, analfabeto, camponês) expressar honestamente um modo de vida natural, enquanto a música clássica parecia superficial e decadente. Do mesmo modo, essa visão frequentemente retratava os povos indígenas como ""selvagens nobres"", vivendo vidas autênticas e sem mácula, sem complicações e sem corrupção pelos sistemas capitalistas altamente estratificados do Ocidente.",Que tipo de música os críticos associaram à alta cultura corrupta?,"{'answer_start': [309], 'text': ['música folclórica']}"
1,57271d57708984140094d9f1,Comcast,"Em janeiro de 2015, um cliente chamado Ricardo Brown recebeu uma fatura da Comcast com o nome alterado para ""Imbecil marrom"". A esposa de Brown, Lisa, acreditava que um funcionário da Comcast mudou o nome em resposta ao pedido dos Browns de cancelar seu serviço a cabo, um incidente no qual ela foi recusada, a menos que pagasse uma taxa de US $ 60 e foi encaminhada para um especialista em retenção. A Comcast se recusou a corrigir o nome em sua conta depois de chamar a atenção de vários pontos de atendimento ao cliente da empresa, explicando que Ricardo é o nome legal do cliente, então os Browns se voltaram para o advogado do consumidor Christopher Elliott. Elliott postou os fatos do incidente, juntamente com uma cópia do projeto, em seu blog. Pouco tempo depois, Elliott entrou em contato com a Comcast e a Comcast ofereceu aos Browns um pedido de desculpas, um reembolso de US $ 60 e uma promessa de rastrear e demitir o funcionário responsável. Os Browns solicitaram um reembolso total por sua experiência negativa e a Comcast concordou em reembolsar a família nos últimos dois anos de serviço e fornecer os próximos dois anos de serviço sem nenhum custo. A Comcast divulgou um comunicado explicando: ""Conversamos com nosso cliente e pedimos desculpas por essa mudança de nome completamente inaceitável e inapropriada. Temos tolerância zero para esse tipo de comportamento desrespeitoso e estamos conduzindo uma investigação completa para determinar o que aconteceu. Estamos trabalhando com nosso cliente a corrigir isso e tomará as medidas necessárias para impedir que isso aconteça novamente "".","Qual nome foi listado na fatura do cliente da Comcast, Ricardo Brown, em janeiro de 2015?","{'answer_start': [108], 'text': ['""Imbecil marrom""']}"
2,5727ee953acd2414000df022,Northwestern_University,"A Northwestern é de propriedade privada e é governada por um Conselho de Administração nomeado. O conselho, composto por 70 membros e a partir de 2011, presidido por William A. Osborn '69, delega seu poder a um presidente eleito para servir como diretor executivo da universidade. A Northwestern teve dezesseis presidentes em sua história (excluindo presidentes interinos), o atual presidente, Morton O. Schapiro, economista, sucedeu a Henry Bienen, cujo mandato de 14 anos terminou em 31 de agosto de 2009. O presidente tem uma equipe de vice-presidentes , diretores e outros assistentes para assuntos administrativos, financeiros, professores e estudantes. Daniel I. Linzer, reitor desde setembro de 2007, atua sob o presidente como diretor acadêmico da universidade a quem reitores de todas as escolas acadêmicas, líderes de unidades interdisciplinares e presidentes do comitê permanente do corpo docente.","Quantos presidentes, excluindo os intervalos, tiveram a Northwestern?","{'answer_start': [301], 'text': ['dezesseis']}"
3,570b2ce2ec8fbc190045b8ba,Xbox_360,"O Kinect é uma ""experiência de jogo e entretenimento sem controle"" para o Xbox 360. Foi anunciado pela primeira vez em 1º de junho de 2009 na Ele

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/155 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > max_length:
        break
example = datasets["train"][i]

print(example)

len(tokenizer(example["question"], example["context"])["input_ids"])

{'id': '5735aaaedc94161900571ef6', 'title': 'Kathmandu', 'context': 'A cidade geralmente tem um clima com dias quentes, seguido de noites e manhãs frias. Prevê-se um clima imprevisível, uma vez que as temperaturas podem cair para 1 ° C (34 ° F) ou menos durante o inverno. Durante uma frente fria de 2013, as temperaturas de inverno de Katmandu caíram para -4 ° C (25 ° F), e a temperatura mais baixa foi registrada em 10 de janeiro de 2013, a -9,2 ° C (15,4 ° F). As chuvas são principalmente baseadas em monções (cerca de 65% do total concentrado durante os meses das monções de Junho a agosto) e diminuem substancialmente (100 a 200 cm) do leste do Nepal ao oeste do Nepal. As chuvas foram registradas em cerca de 1.400 milímetros (55,1 polegadas) no vale de Katmandu e em média 1.407 milímetros (55,4 polegadas) na cidade de Katmandu. Em média, a umidade é de 75%. O gráfico abaixo é baseado em dados do Bureau de Padrões e Meteorologia do Nepal, "Meteorologia Meteorológica" de 2005. O gráfico f

407

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Parameter 'function'=<function prepare_train_features at 0x7f26170cc200> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/88 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

In [ ]:
file_name = str(model_checkpoint).replace('/','-') + '-ric_finetuned-squad-v1.1-pt'

In [ ]:
import transformers
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch_xla.distributed.xla_multiprocessing as xmp

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

WRAPPED_MODEL = xmp.MpModelWrapper(model)

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-large-portuguese-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint a

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

In [ ]:
def training(model):
    args = TrainingArguments(
        f"{file_name}",
        num_train_epochs=4.0,
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        weight_decay=0.001,
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        #save_total_limit=1
    )
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        data_collator=data_collator,
        tokenizer=tokenizer,
    )
    trainer.place_model_on_device = False
    trainer.train()
    trainer.save_model("model_trained/")
    tokenizer.save_pretrained("model_trained/")

In [ ]:
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import warnings

warnings.filterwarnings("ignore")

def _mp_fn(index):
    device = xm.xla_device()

    model = WRAPPED_MODEL.to(device)

    training(model)

xmp.spawn(_mp_fn, nprocs=8, start_method="fork")

***** Running training *****
  Num examples = 89239
  Num Epochs = 4
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 11156


Epoch,Training Loss,Validation Loss
1,1.058600,1.096578
2,0.785400,1.196258
3,0.606500,1.389516
4,0.472400,1.652878


***** Running Evaluation *****
  Num examples = 10917
  Batch size = 4
Saving model checkpoint to neuralmind-bert-large-portuguese-cased-ric_finetuned-squad-v1.1-pt/checkpoint-2789
Configuration saved in neuralmind-bert-large-portuguese-cased-ric_finetuned-squad-v1.1-pt/checkpoint-2789/config.json
Model weights saved in neuralmind-bert-large-portuguese-cased-ric_finetuned-squad-v1.1-pt/checkpoint-2789/pytorch_model.bin
tokenizer config file saved in neuralmind-bert-large-portuguese-cased-ric_finetuned-squad-v1.1-pt/checkpoint-2789/tokenizer_config.json
Special tokens file saved in neuralmind-bert-large-portuguese-cased-ric_finetuned-squad-v1.1-pt/checkpoint-2789/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 10917
  Batch size = 4
Saving model checkpoint to neuralmind-bert-large-portuguese-cased-ric_finetuned-squad-v1.1-pt/checkpoint-5578
Configuration saved in neuralmind-bert-large-portuguese-cased-ric_finetuned-squad-v1.1-pt/checkpoint-5578/config.json
Model 

ProcessExitedException: ignored

In [ ]:
import pathlib
from pathlib import Path

dest = pathlib.Path.cwd()
fname_HF = 'model_trained'

path_to_model = dest/fname_HF

In [ ]:
from transformers import pipeline

model_qa = str(path_to_model)
nlp = pipeline("question-answering", model=model_qa)

In [ ]:
context = r"""
Integralização de um curso é quando o estudante cumpre todas as exigências fixadas no Catálogos de cursos.
O sistema de integralização adotado pela Universidade é o de crédito e matrícula por disciplina.
O relatório de Integralização Curricular é publicado na web semestralmente, constando a relação de disciplinas cursadas com suas respectivas informações - se é obrigatória, eletiva ou extracurricular, créditos e nota obtida -, as disciplinas em que o estudante está matriculado atualmente, as disciplinas que ainda precisam ser cursadas e o prazo máximo para conclusão do curso, dentre outras informações.
O estudante pode consultar esse documento no site da DAC, em Integralização Curricular, após logar em Serviços Acadêmicos.
Prazo mínimo e máximo são os tempos mínimo e máximo que o estudante dispõe para a conclusão do curso. O tempo mínimo é igual ao número de períodos fixados pelas Unidades de Ensino na proposta para o cumprimento do Currículo Pleno. O tempo máximo corresponde à soma do tempo mínimo e mais 50% desse período (exemplo: para um curso de 08 semestres de período mínimo, o prazo máximo seria de 12 semestres).
Do tempo máximo de integralização deve ser subtraído o tempo referente a aproveitamento de estudos, dispensas de disciplinas e teste de proficiência. Isso quer dizer que o período máximo pode ser menor do que o estipulado inicialmente quando o estudante solicitar aproveitamento e/ou teste de proficiência.
Caso o estudante solicite trancamento de semestre, o prazo máximo de integralização será acrescido do mesmo número de períodos letivos em que a matrícula esteve trancada. Isso quer dizer que o período máximo será maior que o estipulado inicialmente quando o estudante solicitar trancamento de semestre.
O estudante pode verificar seu prazo de integralização no site da DAC, em Integralização Curricular, após logar em Serviços Acadêmicos.
"""

In [ ]:
question = "O que tem no relatório de integralização?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
context = r"""
Certificado de Estudos é o documento que atesta a conclusão de um elenco de disciplinas integrantes de um ramo específico do conhecimento.
O oferecimento de Certificados de Estudos bem como a definição do elenco de disciplinas que os compõem ficam a critério da Coordenadoria de Curso responsável pelas disciplinas envolvidas.
Os Certificados de Estudos são emitidos pela Diretoria Acadêmica (DAC) quando solicitados pelos estudantes regulares ou especiais que forem aprovados em todas as disciplinas exigidas.
Mais informações podem ser obtidas no Regimento Geral dos cursos de Graduação.
Procedimentos: Para solicitar o Certificado de Estudos, o estudante deve protocolar seu pedido pelo sistema e-DAC, utilizando o assunto 'Certificado de Estudos' e citando, no campo de texto, o nome do certificado, o catalogo do qual o certificado faz parte(curso/ano) e as disciplinas foram cursadas para este fim.
Caso seja necessário, a Diretoria Acadêmica(DAC) irá solicitar que a Coordenadoria de Graduação responsável emita um ofício, validando o cumprimento das exigidas. Constatando o cumprimento das exigências, a DAC emitirá o Certificado e avisará quando o mesmo estiver disponível para a retirada.
"""

In [ ]:
question = "Como faço para pedir um certificado de estudos?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
context = r"""
Para efeito de cálculo de carga horária, serão atribuídas a cada disciplina unidades de crédito, sendo que cada unidade de crédito corresponde a 15 (quinze) horas/aula, com duração de 15 (quinze) semanas, para as disciplinas semestrais.
O total de créditos a ser cumprido pelo estudante para conclusão do curso será estabelecido pelo Catálogo dos Cursos de Graduação.
Pré-requisitos são condições consideradas indispensáveis para matrícula em disciplinas, podendo ser classificado como:
Pré-requisito pleno: disciplina ou o conjunto de disciplinas em que o estudante deve obter aprovação para matricular-se em outra disciplina.
Pré-requisito parcial: a disciplina ou o conjunto de disciplinas em que o estudante deve obter frequência mínima e média final maior ou igual a 3,0 (três) para matricular-se em outra disciplina.
Pré-requisito especial: condição não atrelada a disciplinas específicas, podendo ser AA4nn - (pela exigência de Coeficiente de Progressão maior ou igual a 0,xx) ou AA200 (por autorização da Coordenadoria que a oferece).
Não é autorizada a utilização da AA200 para disciplinas obrigatórias, devendo os casos excepcionais ser devidamente justificados e remetidos à CCG para apreciação Todos os pré-requisitos exigidos para as disciplinas do curso de graduação constam no Currículo Pleno seguido pelo estudante.
"""

In [ ]:
question = "O que é um pré requisito parcial?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

In [ ]:
!cp -r model_trained/ drive/MyDrive/model_trained_v30/